<a href="https://colab.research.google.com/github/carsofferrei/04_data_processing/blob/main/spark_streaming/example_2_rate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Usecase 2
- Reading data from "rate"
- Aggregating data by window time
- Checking results from query in memory

# Setting up PySpark

In [1]:
%pip install pyspark

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('local').appName('Test streaming').getOrCreate()

# Write output in memory

In [3]:
import pyspark.sql.functions as F

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load() #Escreve 10 ficheiros por segundo

# transform
transformed = stream1.withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count() #Intervalos de 5 em 5 segundos

# write stream in memory
query = (agg.writeStream
.format('memory')
.queryName('my_query')
.outputMode('complete')
.start()
)

In [30]:
spark.sql("select * from my_query order by window desc").show(10,False)

+------------------------------------------+-----+
|window                                    |count|
+------------------------------------------+-----+
|{2024-11-23 15:50:05, 2024-11-23 15:50:10}|30   |
|{2024-11-23 15:50:00, 2024-11-23 15:50:05}|50   |
|{2024-11-23 15:49:55, 2024-11-23 15:50:00}|50   |
|{2024-11-23 15:49:50, 2024-11-23 15:49:55}|50   |
|{2024-11-23 15:49:45, 2024-11-23 15:49:50}|50   |
|{2024-11-23 15:49:40, 2024-11-23 15:49:45}|50   |
|{2024-11-23 15:49:35, 2024-11-23 15:49:40}|10   |
+------------------------------------------+-----+



In [31]:
query.stop()

# Write output as parquet

In [32]:
!rm -rf content/output

In [33]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame

def save_parquet(df, batch_id):
  (df
   .withColumn("batch_id",F.lit(batch_id))
   .withColumn("load_time",F.current_timestamp())
   .write.mode("append")
   .parquet("content/output/rate_parquet")
  )

# read stream
stream1 = spark.readStream.format("rate").option("rowsPerSecond", 10).load()

# transform
transformed = stream1.withWatermark("timestamp", "5 seconds").withColumn("minute", F.minute("timestamp"))
agg = transformed.groupBy(F.window(transformed.timestamp, "5 seconds")).count()

# write stream as parquet with foreachBatch
query = (agg.writeStream
.option('checkpointLocation', 'content/output/checkpoint')
.trigger(processingTime='20 seconds') # se esta janela for menor que os 5 segundos pode gerar duplicados
.outputMode('append')
.foreachBatch(save_parquet) # Dentro desta função passo a lógica de escrita através de um método
.start()
)


In [39]:
result = spark.read.format("parquet").load("content/output/rate_parquet/")
result.sort(F.asc("window")).show(100, False)

# load_time - exatamente o momento em que escreveu

+------------------------------------------+-----+--------+--------------------------+
|window                                    |count|batch_id|load_time                 |
+------------------------------------------+-----+--------+--------------------------+
|{2024-11-23 15:55:25, 2024-11-23 15:55:30}|14   |2       |2024-11-23 15:56:00.250838|
|{2024-11-23 15:55:30, 2024-11-23 15:55:35}|50   |2       |2024-11-23 15:56:00.250838|
|{2024-11-23 15:55:35, 2024-11-23 15:55:40}|50   |3       |2024-11-23 15:56:20.296245|
|{2024-11-23 15:55:40, 2024-11-23 15:55:45}|50   |3       |2024-11-23 15:56:20.296245|
|{2024-11-23 15:55:45, 2024-11-23 15:55:50}|50   |3       |2024-11-23 15:56:20.296245|
|{2024-11-23 15:55:50, 2024-11-23 15:55:55}|50   |4       |2024-11-23 15:56:40.280023|
|{2024-11-23 15:55:55, 2024-11-23 15:56:00}|50   |4       |2024-11-23 15:56:40.280023|
|{2024-11-23 15:56:00, 2024-11-23 15:56:05}|50   |4       |2024-11-23 15:56:40.280023|
|{2024-11-23 15:56:05, 2024-11-23 15:56:10}

In [40]:
query.stop()

ERROR:py4j.clientserver:There was an exception while executing the Python Proxy on the Python Side.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/py4j/clientserver.py", line 617, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 120, in call
    raise e
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/utils.py", line 117, in call
    self.func(DataFrame(jdf, wrapped_session_jdf), batch_id)
  File "<ipython-input-33-bd46483e2cb2>", line 9, in save_parquet
    .parquet("content/output/rate_parquet")
  File "/usr/local/lib/python3.10/dist-packages/pyspark/sql/readwriter.py", line 1721, in parquet
    self._jwrite.parquet(path)
  File "/usr/local/lib/python3.10/dist-packages/py4j/java_gateway.py", line 1322, in __call__
    return_value = get_return_value(
  File "/usr/local/lib/python3.10/dist-packages/pyspark/errors/exceptions/captured.py

# Enrich data with faker

In [41]:
!pip install faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00


In [42]:
!rm -rf content/output/events

In [43]:
import pyspark.sql.functions as F
from pyspark.sql import DataFrame
from faker import Faker

def insert_into_table(df, batch_id):
  fake = Faker()
  new_columns = {
      'name': F.lit(fake.name()),
      'address': F.lit(fake.address()),
      'email': F.lit(fake.email()),
      'dob': F.lit(fake.date_of_birth()),
      'phone': F.lit(fake.phone_number())
  }
  df = df.withColumns(new_columns)
  df.write.mode("append").format("parquet").save("content/output/events")

# read stream
df_stream = spark.readStream.format("rate").option("rowsPerSecond", 1).load()

# write stream
query = (df_stream.writeStream
.outputMode('append')
.trigger(processingTime='1 seconds')
.foreachBatch(insert_into_table)
.start()
)

In [45]:
query.stop()


In [46]:
spark.read.parquet("content/output/events").show(100, False)

+-----------------------+-----+-------------------------+-------------------------------------------------------------+-----------------------------+----------+----------------------+
|timestamp              |value|name                     |address                                                      |email                        |dob       |phone                 |
+-----------------------+-----+-------------------------+-------------------------------------------------------------+-----------------------------+----------+----------------------+
|2024-11-23 16:02:24.176|45   |Annette Cummings         |049 Hendrix Greens Suite 151\nSouth Brandonville, NH 18912   |traceytrujillo@example.com   |1912-08-03|001-852-431-0385x28876|
|2024-11-23 16:01:59.176|20   |Mrs. Crystal Johnston DDS|268 Brandy Ridge Apt. 864\nNorth Brookemouth, MA 55871       |cgomez@example.com           |1965-06-24|001-510-544-3931x5468 |
|2024-11-23 16:03:11.176|92   |Raymond Johnson          |082 Price Gateway Apt. 

In [ ]:
query.stop()